# Environmental sustainability of digital content consumption

### This notebook contains functions to perform LCA calculations

In [12]:
def multi_lcia(activity, amount, lcia_methods):
    '''
    This function calculates the impact accross a range of categories
    '''
    lca = bw.LCA({activity.key: amount})
    lca.lci()
    multi_lcia_results = dict()
    for impact in lcia_methods:
        lca.switch_method(lcia_methods[impact])
        lca.lcia()
        multi_lcia_results[impact] = lca.score
    return multi_lcia_results

In [3]:
def multi_lcia_MonteCarlo(activity, iterations, lcia_methods):
    '''
    This function performs Monte Carlo simulation accross a range of categories
    '''
    # Create a MonteCarloLCA object with functional unit but no method. 
    MC_LCA = bw.MonteCarloLCA({activity: 1})
    
    # Run .lci to build the A and B matrices (and hence fix the indices of our matrices)
    MC_LCA.lci()
    
    # Iterate over the methods and stores the C matrix in a dictionary
    C_matrices = dict()
    for method in lcia_methods:
        MC_LCA.switch_method(lcia_methods[method])
        C_matrices[method] = MC_LCA.characterization_matrix
        
    # Create a dictionary to store the results
    results_MC = np.empty((len(lcia_methods), iterations))
   
    # Populate results dictionary using a for loop over the number of MonteCarlo iterations required 
    # Include a nested for loop over the methods and multiply the characterization matrix with the inventory.       
    for iteration in range(iterations):
        next(MC_LCA)
        for method_index, method in enumerate(lcia_methods):
            results_MC[method_index, iteration] = (C_matrices[method] * MC_LCA.inventory).sum()
       
    results_MC_dict = dict()
    i_count = 0
    for method in lcia_methods:
        results_MC_dict[method] = results_MC[i_count]
        i_count += 1

    return results_MC_dict

def stats_analysis_mc_results(mc_results_df):
    stats_results = mc_results_df.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])
    stats_results.loc['CV'] = stats_results.loc['std'] * 100 / stats_results.loc['mean']
    stats_results = stats_results.T.drop('count', axis=1)
    return stats_results

In [13]:
def lcia_per_user_and_loc(users, locations, db, lcia_methods):
    '''
    This function calculates environmental impacts per user type and location
    '''
    lcia_results = dict()
    for user in users:
        lcia_results[user] = {}
        for loc in locations:
            ds = [ds for ds in bw.Database(db) if user.lower() in ds['name'] and ds['location'] == loc][0]
            multi_lcia_results = multi_lcia(ds, 1, lcia_methods)
            lcia_results[user][locations[loc]] = multi_lcia_results
    return lcia_results

In [1]:
def impacts_vs_carrying_capacity(users, locations, lcia_results, limits_df):
    '''
    This function assess the impacts against the per capita carrying capacity
    '''
    contributions = dict()
    for user in users:
        contributions[user] = {}
        for loc in locations:
            contributions[user][locations[loc]] = {}
            for impact in lcia_results[user][locations[loc]]:
                if impact == 'Carbon footprint':
                    pass
                else:
                    contributions[user][locations[loc]][impact] = lcia_results[user][locations[loc]][impact] * 100 / limits_df.to_dict()[impact]
    return contributions

In [ ]:
def internet_system_contributions(activity, lcia_methods, activity_amount=1):
    '''
    This function computes the contribution of each Internet system component to the total impact
    Based on: https://github.com/brightway-lca/brightway2/blob/master/notebooks/Contribution%20analysis%20and%20comparison.ipynb
    '''
    
    # Disaggregation labels:
    system_components = ['End-user devices | operation',
                         'End-user devices | embodied', 
                         'CPE | operation',
                         'CPE | embodied', 
                         'Access network | operation', 
                         'Core network | operation', 
                         'Data centres | operation',
                         'Data centres | embodied']
    
    
    # 2-levels cumulative LCI requirements
    lci_requirements = {}
    for oa in activity.technosphere():
        oa_amount = activity_amount * oa['amount']
        for system in bw.get_activity(oa.input).technosphere():
            if system.input not in lci_requirements:
                lci_requirements[system.input] = oa_amount * system['amount']
            else:
                lci_requirements[system.input] += oa_amount * system['amount']

    # Open dictionary and compute impacts
    system_contributions = dict()
    for subsystem in system_components:
        system_contributions[subsystem] = {}
        for impact in lcia_methods:
            system_contributions[subsystem][impact] = 0

    for system in lci_requirements:
        system_amount = lci_requirements[system]

        # Impacts embodied in end-user devices:
        if system['name'] in ['market for consumer electronics, mobile device, smartphone',
                              'market for consumer electronics, mobile device, tablet',
                              'market for computer, laptop',
                              'market for computer, desktop, without screen',
                              'market for display, liquid crystal, 17 inches',
                              'market for used liquid crystal display',
                              'market for television']:
            multi_lcia_results = multi_lcia(system, system_amount, lcia_methods)
            for impact in multi_lcia_results:
                system_contributions['End-user devices | embodied'][impact] += multi_lcia_results[impact]
        
        # Impacts operation of transmission network:
        elif system['name'] == 'access network, internet access':
            multi_lcia_results = multi_lcia(system, system_amount, lcia_methods)
            for impact in multi_lcia_results:
                system_contributions['Access network | operation'][impact] += multi_lcia_results[impact]
        elif 'core network, internet access' in system['name']:
            multi_lcia_results = multi_lcia(system, system_amount, lcia_methods)
            for impact in multi_lcia_results:
                system_contributions['Core network | operation'][impact] += multi_lcia_results[impact]
            
        # System parts with embodied emissions
        elif 'costumer premise equipment, internet access' in system['name']:
            for subsystem in system.technosphere():
                subsystem_amount = system_amount * subsystem['amount']
                multi_lcia_results = multi_lcia(subsystem.input, subsystem_amount, lcia_methods)
                for impact in multi_lcia_results:
                    if subsystem.input['name'] == 'market for internet access equipment':
                        system_contributions['CPE | embodied'][impact] += multi_lcia_results[impact]
                    else:
                        system_contributions['CPE | operation'][impact] += multi_lcia_results[impact]
                                            
        elif 'market for data centers, internet access' in system['name']:
            for subsystem in system.technosphere():
                subsystem_amount = system_amount * subsystem['amount']
                multi_lcia_results = multi_lcia(subsystem.input, subsystem_amount, lcia_methods)
                for impact in multi_lcia_results:
                    if subsystem.input['name'] in ['market for computer, desktop, without screen', 'market for hard disk drive, for desktop computer', 'market for chassis, internet access equipment']:
                        system_contributions['Data centres | embodied'][impact] += multi_lcia_results[impact]
                    else:
                        system_contributions['Data centres | operation'][impact] += multi_lcia_results[impact]
        
        else:
            multi_lcia_results = multi_lcia(system, system_amount, lcia_methods)
            for impact in multi_lcia_results:
                system_contributions['End-user devices | operation'][impact] += multi_lcia_results[impact]
    
    return system_contributions

In [19]:
def user_devices_embodied_impacts(activity, lcia_methods, activity_amount=1):
    '''
    This function computes the contribution of embodied impacts of electronic devices to the total impact
    '''
    
    user_devices_inventories = ['market for consumer electronics, mobile device, smartphone',
                                'market for consumer electronics, mobile device, tablet',
                                'market for computer, laptop',
                                'market for computer, desktop, without screen',
                                'market for display, liquid crystal, 17 inches',
                                'market for used liquid crystal display',
                                'market for television']
    
    user_devices_embodied_lcia = dict()
    user_devices_embodied_lcia['Smartphone'] = {}
    user_devices_embodied_lcia['Tablet'] = {}
    user_devices_embodied_lcia['Laptop'] = {}
    user_devices_embodied_lcia['Desktop computer'] = {}
    user_devices_embodied_lcia['Television'] = {}
    for device in user_devices_embodied_lcia:
        for impact in lcia_methods:
            user_devices_embodied_lcia[device][impact] = 0
    
    for oa in activity.technosphere():
        oa_amount = oa['amount']
        for system in bw.get_activity(oa.input).technosphere():
            system_amount = oa_amount * system['amount']
            if system.input['name'] in user_devices_inventories:
                multi_lcia_results = multi_lcia(system.input, system_amount, lcia_methods)
                for impact in multi_lcia_results:
                    if system.input['name'] == user_devices_inventories[0]:
                        user_devices_embodied_lcia['Smartphone'][impact] += multi_lcia_results[impact]
                    elif system.input['name'] == user_devices_inventories[1]:
                        user_devices_embodied_lcia['Tablet'][impact] += multi_lcia_results[impact]                    
                    elif system.input['name'] == user_devices_inventories[2]:
                        user_devices_embodied_lcia['Laptop'][impact] += multi_lcia_results[impact] 
                    elif system.input['name'] == user_devices_inventories[6]:
                        user_devices_embodied_lcia['Television'][impact] += multi_lcia_results[impact] 
                    else:
                        user_devices_embodied_lcia['Desktop computer'][impact] += multi_lcia_results[impact] 
    return user_devices_embodied_lcia

In [18]:
def user_devices_operational_impacts(activity, lcia_methods, activity_amount=1):
    '''
    This function computes the contribution of operational impacts of electronic devices to the total impact
    '''
    
    user_devices_operational_lcia = dict()
    user_devices_operational_lcia['Smartphone'] = {}
    user_devices_operational_lcia['Tablet'] = {}
    user_devices_operational_lcia['Laptop'] = {}
    user_devices_operational_lcia['Desktop computer'] = {}
    user_devices_operational_lcia['Television'] = {}
    for device in user_devices_operational_lcia:
        for impact in lcia_methods:
            user_devices_operational_lcia[device][impact] = 0
    
    for oa in activity.technosphere():
        oa_amount = oa['amount']
        for system in bw.get_activity(oa.input).technosphere():
            if 'electricity' in system['name']:
                system_amount = oa_amount * system['amount']
                multi_lcia_results = multi_lcia(system.input, system_amount, lcia_methods)
        
        for impact in multi_lcia_results:
            if 'smartphone' in oa['name']:
                user_devices_operational_lcia['Smartphone'][impact] += multi_lcia_results[impact]
            elif 'tablet' in oa['name']:
                user_devices_operational_lcia['Tablet'][impact] += multi_lcia_results[impact]
            elif 'laptop' in oa['name']:
                user_devices_operational_lcia['Laptop'][impact] += multi_lcia_results[impact]
            elif 'desktop computer' in oa['name']:
                user_devices_operational_lcia['Desktop computer'][impact] += multi_lcia_results[impact]
            elif 'television' in oa['name']:
                user_devices_operational_lcia['Television'][impact] += multi_lcia_results[impact]

    return user_devices_operational_lcia

In [20]:
def digital_content_constributions(activity, lcia_methods, digital_content, activity_amount=1):
    '''
    This function computes the contribution of digital contents to the total impact
    '''
    content_contributions = dict()
    for exc in activity.technosphere():
        exc_amount = activity_amount * exc['amount']
        multi_lcia_results = multi_lcia(exc.input, exc_amount, lcia_methods)
        content_contributions.update({exc['name']: multi_lcia_results})
        
    content_contributions_group = {}
    for element in digital_content:
        content_contributions_group[element] = {}
        for impact in lcia_methods.keys():
            content_contributions_group[element][impact] = 0
            for element_i in content_contributions:
                if element.lower() in element_i:
                    content_contributions_group[element][impact] += content_contributions[element_i][impact]
    return content_contributions_group

In [ ]:
def read_presamples_scenario_data(scenario_file, Ecoinvent_db, Other_db):
    """
    This function reads the scenario data from an excel file and prepares it into a dataframe for being used with Presamples.
    Secondly, it adds the bw codes for the involved activities.
    The dictionary map_bw_keys provides these codes, it needs to be generated beforehand with the involved databases.    
    """

    # Create mapping of BW codes for involved databases
    map_bw_keys = dict()
    for db in [Ecoinvent_db, Other_db]:
        db_obj = bw.Database(db)
        for ds in db_obj:
            map_bw_keys[(ds['reference product'], ds['name'], ds['location'])] = ds.key
    
    scenariodata_df = pd.read_excel(scenario_file) # Import data
    scenariodata_df = scenariodata_df.dropna(how="all") # Delete empty rows
    
    scenario_label = list(scenariodata_df.columns)[9:] # Get label of scenarios
    
    # add the bw code to your scenario DF (input = process, output = to_process)
    scenariodata_df["input"] = [map_bw_keys[(row["from_reference_product"], row["from_process"], row["from_location"])] 
                                for index, row in scenariodata_df.iterrows()]
    scenariodata_df["output"] = [map_bw_keys[(row["to_reference product"], row["to_process"], row["to_location"])] 
                                 for index, row in scenariodata_df.iterrows()]

    return scenario_label, scenariodata_df

In [ ]:
def make_presample_package(scenariodata_df, scenario_label, matrixlabel, ps_packagename):
    """
    This function prepares a Presamples package out of the scenario data if.
    """
        
    # select needed data
    samples = scenariodata_df[scenario_label].values
    indices = [(row['input'], row['output'], row['from_type']) for i, row in scenariodata_df.iterrows()] 

    # Generate PS data in PSpackage
    data = [(samples, indices, matrixlabel)]
    
    ps_id, ps_filepath = ps.create_presamples_package(matrix_data=data, name=ps_packagename, seed="sequential") 
    
    print("\n ps_id, filepath:", ps_id, ps_filepath)
    
    return samples, indices, ps_filepath

In [ ]:
def calculate_impacts_with_presamples(input_dbs, DBS, LCIA_METHODS, input_carrying_capacity):
       ecoinvent_db = DBS[input_dbs[0]]
       internet_db = DBS[input_dbs[1]]

       # Read the excel file to get the scenario data
       scenario_label, scenariodata_df = read_presamples_scenario_data(sensitivity_analysis_data, ecoinvent_db, internet_db)

       # Create PS package with indices + samples matrix
       samples_v1, indices_v1, ps_filepath_v1 =  make_presample_package(scenariodata_df, scenario_label, "technosphere", "ps_v1")

       # Get activity for assessment and calculate impacts
       activity_for_ps = [ds for ds in bw.Database(internet_db) if 'medium technology user' in ds['name'] and ds['location'] == 'GLO'][0]
       activity_for_ps_lca = bw.LCA({activity_for_ps:1}, presamples=[ps_filepath_v1])

       sensitivity_analysis_lcia = dict()
       for impact in LCIA_METHODS:
              sensitivity_analysis_lcia[impact] = {}

       scenario_lcia = dict()
       for i in range(len(scenario_label)): # Scenarios
              if i == 0: # Don't update the first time around, since indexer already at 0th column
                     activity_for_ps_lca.lci() # Builds matrices
                     multi_lcia_results = dict()
                     for impact in LCIA_METHODS:
                            activity_for_ps_lca.switch_method(LCIA_METHODS[impact])
                            activity_for_ps_lca.lcia()
                            multi_lcia_results[impact] = activity_for_ps_lca.score
              else:
                     activity_for_ps_lca.presamples.update_matrices() # Move to next column and update matrices
                     activity_for_ps_lca.redo_lci()
                     multi_lcia_results = dict()
                     for impact in LCIA_METHODS:
                            activity_for_ps_lca.switch_method(LCIA_METHODS[impact])
                            activity_for_ps_lca.lcia()
                            multi_lcia_results[impact] = activity_for_ps_lca.score
                     
              scenario_lcia[scenario_label[i]] = multi_lcia_results

              for impact in LCIA_METHODS:
                     sensitivity_analysis_lcia[impact][scenario_label[i]] = scenario_lcia[scenario_label[i]][impact]

       sensitivity_analysis_lcia_df = pd.DataFrame(sensitivity_analysis_lcia).T
       
       # Divide by carrying capacity
       sensitivity_analysis_lcia_carrying_capacity = sensitivity_analysis_lcia_df.iloc[1:].divide(input_carrying_capacity, axis=0).multiply(100)

       return sensitivity_analysis_lcia_df, sensitivity_analysis_lcia_carrying_capacity